In [2]:
# Install the Libraries
!pip install -q langchain torch transformers sentence-transformers datasets faiss-cpu unstructured chromadb gradio langchain-community langchain-huggingface

ERROR: Cannot uninstall nest-asyncio 1.5.8, RECORD file not found. You might be able to recover from this via: 'pip install --force-reinstall --no-deps nest-asyncio==1.5.8'.


In [3]:
# Import Libraries
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.vectorstores import FAISS
from transformers import GPT2LMHeadModel, GPT2Tokenizer, pipeline
from langchain import HuggingFacePipeline, LLMChain
from langchain.chains import RetrievalQA, StuffDocumentsChain
from langchain.prompts import PromptTemplate
import gradio as gr
import faiss

2024-06-06 23:37:46.008389: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-06 23:37:46.008513: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-06 23:37:46.215180: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
# Load Documents from Google Drive
loader = DirectoryLoader('/kaggle/input/agri-html')
data = loader.load()

Exception ignored in: <function _xla_gc_callback at 0x7d979e61a200>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/jax/_src/lib/__init__.py", line 98, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


In [ ]:
import random
# Split Documents into Chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""],
    is_separator_regex=True
)
documents = text_splitter.split_documents(data)

print(random.choice(documents).page_content)

In [ ]:
import torch
import torch.nn as nn
from transformers import (
    pipeline,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoConfig,
)
from langchain.llms import HuggingFacePipeline

import warnings
from IPython.display import Markdown
import re
import random
import pandas as pd

In [ ]:
# Text Embedding Model
# embeddings = HuggingFaceEmbeddings(model_name="distilbert-base-uncased")

# from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()
# ### Using embeddings by MPNET
# model_name = "sentence-transformers/all-mpnet-base-v2"
# # model_kwargs = {"device": "cuda" if torch.cuda.is_available() else "cpu"}
# encode_kwargs = {"normalize_embeddings": False}
# embeddings = HuggingFaceEmbeddings(
#     model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
# )

In [ ]:
# Create the Vector Store with Chroma
db = Chroma.from_documents(documents, embeddings, persist_directory="/path/to/vectorstore")

# Persist the database
db.persist()

In [ ]:
# Test Similarity Search
question = "Most Popular Agricultural Product in Ethiopia"
search_docs = db.similarity_search(question)
print(f"Sample Search Result: {search_docs[0].page_content}")

In [ ]:
# Query to retrieve similar chunks
query = "What is the Most Popular Agricultural Product in Ethiopia?"

# Retrieve similar chunks based on relevance. We only retrieve 'k' most similar chunks
similar_chunks = db.similarity_search_with_relevance_scores(query, k=3)

# Format document to text format
retrieved_text = [chunk[0].page_content for chunk in similar_chunks]
relevance_score = [chunk[1] for chunk in similar_chunks]

# Store and print as a dataframe
retrieved_chunks = pd.DataFrame(
    list(zip(retrieved_text, relevance_score)),
    columns=["Retrieved Chunks", "Relevance Score"],
)
with pd.option_context("display.max_colwidth", None):
    display(retrieved_chunks)

In [ ]:
# # Prepare the LLM Model (distilGPT2)
# tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")
# model = GPT2LMHeadModel.from_pretrained("distilgpt2")
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

model_name = "tiiuae/falcon-7b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
# Get the eos_token_id
eos_token_id = tokenizer.eos_token_id

# Set pad_token_id to eos_token_id
model.config.pad_token_id = eos_token_id


In [ ]:
# # Create Text Generation Pipeline with `max_new_tokens`
# text_generator = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     max_length=1024,  # Adjusted max_length
#     max_new_tokens=200  # Controls the number of tokens generated
# )

text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=False,
    repetition_penalty=1.6,
    temperature=0.3,
    top_k=30
)

In [ ]:
from langchain_huggingface import HuggingFacePipeline

# Create HuggingFacePipeline LLM
llm = HuggingFacePipeline(pipeline=text_generator)
llm.model_id = model_name

In [ ]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [ ]:
# Supress warnings
warnings.filterwarnings("ignore")

prompt = """Use the given context to answer the question.
If you don't know the answer, just say that you don't know. Never Hallucinate or Repeat the answers.
Keep the answer as concise as possible.

Context: {context}

Question: {question}
Answer:
"""

qa_prompt_template = PromptTemplate.from_template(prompt)

In [ ]:
# # Define the RetrievalQ&A chain
# qa_chain = RetrievalQA.from_chain_type(
#     pipeline,
#     retriever=db.as_retriever(search_kwargs={'k': 1}),
#     return_source_documents=True,
#     chain_type="stuff",
#     chain_type_kwargs={"prompt": qa_prompt_template},
# )

In [ ]:
# Create the LLM Chain
llm_chain = LLMChain(
    llm=llm,
    prompt=qa_prompt_template
)

In [ ]:
# Create the StuffDocumentsChain
stuff_docs_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="context"  # Specify the name of the variable that will store the retrieved documents
)

In [ ]:
# Combine into RetrievalQA Chain
qa = RetrievalQA(
    retriever=db.as_retriever(search_kwargs={'k': 1}),
    combine_documents_chain=stuff_docs_chain
)

In [ ]:
# Perform retrieval Q&A
qa_response = qa({"query": "What is the Most Popular Agricultural Product in Ethiopia?"})
Markdown(qa_response["result"])

In [ ]:
# def query_chatbot(question: str) -> str:
#     try:
#         result = qa.run(question)
#         # Extract the answer from the result
#         answer = result.split("Helpful Answer:")[1].strip()
#         return answer
#     except Exception as e:
#         return f"An error occurred: {str(e)}"

In [ ]:
# with gr.Blocks() as demo:
#     gr.Markdown("AASTUChat")
#     question_input = gr.Textbox(label="Ask a Question")
#     answer_output = gr.Textbox(label="Answer", interactive=False)
#     submit_button = gr.Button("Submit")
#     submit_button.click(fn=query_chatbot, inputs=question_input, outputs=answer_output)

# # Launch Gradio App
# demo.launch(share=True)

In [ ]:
# import requests
# from time import sleep

# # Replace 'YOUR_GRADIO_URL' with the actual URL of your Gradio interface
# keepalive_url = f'{YOUR_GRADIO_URL}/keepalive'

# while True:
#   # Send a GET request to the keepalive endpoint every 5 minutes
#   requests.get(keepalive_url)
#   sleep(300)  # Wait 5 minutes between requests
